In [15]:
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

In [16]:
res = requests.get("http://127.0.0.1:8000/dataset-pokemon")
data = pd.DataFrame(res.json())

In [17]:
data

,nome,altura,peso,experiencia_base,tipo_principal,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos
0,bulbasaur,7,69,64,grass,45,49,49,65,65,45,2,86
1,ivysaur,10,130,142,grass,60,62,63,80,80,60,2,83
2,venusaur,20,1000,236,grass,80,82,83,100,100,80,2,96
3,charmander,6,85,62,fire,39,52,43,60,50,65,2,105
4,charmeleon,11,190,142,fire,58,64,58,80,65,80,2,103
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,iron-thorns,16,3030,285,rock,100,134,110,70,84,72,1,75
995,frigibax,5,170,64,dragon,65,75,45,35,45,55,2,35
996,arctibax,8,300,148,dragon,90,95,66,45,65,62,2,36
997,baxcalibur,21,2100,300,dragon,115,145,92,75,86,87,2,53


In [18]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   nome              999 non-null    str  
 1   altura            999 non-null    int64
 2   peso              999 non-null    int64
 3   experiencia_base  999 non-null    int64
 4   tipo_principal    999 non-null    str  
 5   hp                999 non-null    int64
 6   ataque            999 non-null    int64
 7   defesa            999 non-null    int64
 8   ataque_especial   999 non-null    int64
 9   defesa_especial   999 non-null    int64
 10  velocidade        999 non-null    int64
 11  qtd_habilidades   999 non-null    int64
 12  qtd_movimentos    999 non-null    int64
dtypes: int64(11), str(2)
memory usage: 101.6 KB


In [19]:
data.describe()

,altura,peso,experiencia_base,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,11.987988,646.662663,148.064064,69.659660,77.063063,71.871872,69.406406,69.645646,66.560561,2.376376,77.121121
std,12.467587,1181.053028,76.917804,26.564556,29.727773,29.165851,29.364665,26.568031,28.463493,0.733183,26.463427
min,1.000000,1.000000,36.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.000000,1.000000
25%,5.000000,85.000000,66.000000,50.000000,55.000000,50.000000,45.000000,50.000000,45.000000,2.000000,62.000000
50%,10.000000,280.000000,157.000000,66.000000,75.000000,68.000000,65.000000,65.000000,65.000000,3.000000,77.000000
75%,15.000000,660.500000,182.000000,82.000000,100.000000,90.000000,90.000000,85.000000,85.000000,3.000000,92.000000
max,200.000000,9999.000000,608.000000,255.000000,181.000000,230.000000,173.000000,230.000000,200.000000,3.000000,375.000000


In [20]:
# 1. Identificar colunas numéricas automaticamente
colunas_numericas = data.select_dtypes(include=[np.number]).columns.tolist()

# 3. Aplicar a normalização APENAS nas colunas selecionadas
scaler = StandardScaler()

# CORREÇÃO AQUI: Passamos data[cols_para_cluster] em vez de apenas data
data_scaled = scaler.fit_transform(data[colunas_numericas])

# 4. Agora você pode visualizar os dados normalizados sem erro
data_scaled = pd.DataFrame(data_scaled, columns=colunas_numericas)

In [21]:
data_scaled

,altura,peso,experiencia_base,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos
0,-0.400277,-0.489353,-1.093455,-0.928757,-0.944474,-0.784593,-0.150133,-0.174946,-0.757861,-0.513603,0.335683
1,-0.159532,-0.437678,-0.078878,-0.363812,-0.506954,-0.304339,0.360941,0.389925,-0.230606,-0.513603,0.222262
2,0.642949,0.299321,1.143818,0.389448,0.166155,0.381738,1.042372,1.143087,0.472401,-0.513603,0.713752
3,-0.480525,-0.475799,-1.119470,-1.154735,-0.843508,-0.990416,-0.320491,-0.739817,-0.054854,-0.513603,1.054015
4,-0.079284,-0.386851,-0.078878,-0.439138,-0.439643,-0.475859,0.360941,-0.174946,0.472401,-0.513603,0.978401
...,...,...,...,...,...,...,...,...,...,...,...
994,0.321957,2.018987,1.781181,1.142708,1.916237,1.307941,0.020225,0.540557,0.191198,-1.878201,-0.080193
995,-0.560773,-0.403793,-1.093455,-0.175497,-0.069433,-0.921809,-1.172281,-0.928108,-0.406357,-0.513603,-1.592470
996,-0.320029,-0.293667,-0.000833,0.766078,0.603675,-0.201428,-0.831565,-0.174946,-0.160305,-0.513603,-1.554663
997,0.723197,1.231160,1.976292,1.707653,2.286447,0.690472,0.190583,0.615873,0.718453,-0.513603,-0.911945


In [22]:
# 4. Criar 6 Clusters (Um para cada vaga no time)
kmeans = KMeans(n_clusters=6, random_state=18, n_init=10)
data['cluster'] = kmeans.fit_predict(data_scaled)

In [23]:
data

,nome,altura,peso,experiencia_base,tipo_principal,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos,cluster
0,bulbasaur,7,69,64,grass,45,49,49,65,65,45,2,86,5
1,ivysaur,10,130,142,grass,60,62,63,80,80,60,2,83,3
2,venusaur,20,1000,236,grass,80,82,83,100,100,80,2,96,4
3,charmander,6,85,62,fire,39,52,43,60,50,65,2,105,5
4,charmeleon,11,190,142,fire,58,64,58,80,65,80,2,103,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,iron-thorns,16,3030,285,rock,100,134,110,70,84,72,1,75,4
995,frigibax,5,170,64,dragon,65,75,45,35,45,55,2,35,5
996,arctibax,8,300,148,dragon,90,95,66,45,65,62,2,36,0
997,baxcalibur,21,2100,300,dragon,115,145,92,75,86,87,2,53,4


In [24]:
# 5. Montar o "Time Ideal" pegando o melhor de cada cluster
# Vamos critério: o Pokémon com a maior soma de atributos em cada cluster
features = ['hp', 'ataque', 'defesa', 'ataque_especial', 'defesa_especial', 'velocidade']
data['total_stats'] = data[features].sum(axis=1)

In [25]:
data

,nome,altura,peso,experiencia_base,tipo_principal,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos,cluster,total_stats
0,bulbasaur,7,69,64,grass,45,49,49,65,65,45,2,86,5,318
1,ivysaur,10,130,142,grass,60,62,63,80,80,60,2,83,3,405
2,venusaur,20,1000,236,grass,80,82,83,100,100,80,2,96,4,525
3,charmander,6,85,62,fire,39,52,43,60,50,65,2,105,5,309
4,charmeleon,11,190,142,fire,58,64,58,80,65,80,2,103,0,405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,iron-thorns,16,3030,285,rock,100,134,110,70,84,72,1,75,4,570
995,frigibax,5,170,64,dragon,65,75,45,35,45,55,2,35,5,320
996,arctibax,8,300,148,dragon,90,95,66,45,65,62,2,36,0,423
997,baxcalibur,21,2100,300,dragon,115,145,92,75,86,87,2,53,4,600


In [26]:
time_ideal = data.sort_values('total_stats', ascending=False).drop_duplicates('cluster')

In [27]:
time_ideal

,nome,altura,peso,experiencia_base,tipo_principal,hp,ataque,defesa,ataque_especial,defesa_especial,velocidade,qtd_habilidades,qtd_movimentos,cluster,total_stats
492,arceus,32,3200,324,normal,120,120,120,120,120,120,1,173,4,720
889,eternatus,200,9500,345,poison,140,85,95,145,95,130,1,53,2,690
783,kommo-o,16,782,270,dragon,75,110,125,100,105,85,3,107,1,600
718,diancie,7,88,270,rock,50,100,150,100,150,50,1,90,3,600
904,enamorus-incarnate,16,480,116,fairy,74,115,70,135,80,106,2,54,0,580
565,archen,5,95,71,rock,55,112,45,74,45,70,1,75,5,401
